## Tensorflow Object Detection API:

https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/

Interesting link to apply TF Object Detection API and pre-calibrated model from model zoo (wind turbines):

https://medium.com/analytics-vidhya/tensorflow-object-detection-api-tutorial-wind-turbine-detection-using-google-colab-e8e2e120e54e

this is the code: https://github.com/lbborkowski/wind-turbine-detector

Maybe also interesting:
https://github.com/rhammell/planesnet-detector

## YOLO v8: 
https://www.kaggle.com/code/jeffaudi/aircraft-detection-with-yolov8

# Identifying & mapping aircrafts in high-resolution satellite images
## Comparing YOLO v8 and TensorFlow Object Detection API

TensorFlow provides a powerful library called TensorFlow Object Detection API, which can be utilized to achieve this goal. Here's how this notebook is structured:

1. **Prepare Your Dataset**: Obtain a dataset of satellite images that you want to analyze. This dataset should contain the images and annotations for the objects you want to detect in the images (e.g., aircraft, buildings, vehicles, etc.). Annotations should include bounding boxes around the objects and their corresponding class labels.

2. **Install TensorFlow and TensorFlow Object Detection API**: Make sure you have TensorFlow installed, and then set up the TensorFlow Object Detection API. You can find installation instructions in the official TensorFlow Object Detection API repository on GitHub.

3. **Select a Pre-trained Model**: Choose a pre-trained model from the TensorFlow Model Zoo that suits your object detection needs. These pre-trained models are trained on large datasets like COCO and can serve as a starting point for your analysis.

4. **Prepare the Configuration**: Create a configuration file for your selected model. This file defines the model architecture, hyperparameters, and paths to your dataset (training and evaluation data). You'll need to modify the configuration file to match your dataset and requirements.

5. **Training**: Train the model using your prepared dataset and the configuration file. Fine-tune the pre-trained model on your satellite images dataset to detect the objects you're interested in.

6. **Evaluate the Model**: Once training is complete, evaluate the performance of your model on a separate validation dataset. This step is essential to measure how well your model generalizes to unseen data.

7. **Object Detection on New Satellite Images**: After the model has been trained and evaluated, you can use it to perform object detection on new satellite images. The model will predict bounding boxes and class labels for the objects in these images.

8. **Visualize the Results**: Display the results of the object detection model on the satellite images to visualize the locations of detected objects.


Additionally, you may need to implement post-processing techniques to refine the detected bounding boxes and remove false positives or apply custom object filtering strategies to focus only on specific objects of interest.

The TensorFlow Object Detection API documentation provides detailed instructions, tutorials, and examples that can guide you through each step of this process. Make sure to refer to the official documentation to get more detailed and up-to-date information.

In [1]:
# Import the required functions from utils.py
%load_ext autoreload
%autoreload 2

import os
import ast
import PIL
import utils
import shutil
import tqdm.notebook
import numpy as np
import pandas as pd
from pathlib import Path
from config import Config
from IPython.display import display, clear_output
# !pip install imgaug

config = Config()

## Step 1: Load and prepare the dataset

In [2]:
# Define the path to your data folder containing images
IMAGE_PATH = config.image_folder

# Create the dropdown menu
dropdown_menu = utils.create_image_dropdown(IMAGE_PATH)

# Display the dropdown menu
display(dropdown_menu)

Dropdown(description='Select an image:', options=('78400c58-1a7c-4342-a1fb-2117cb7cbc8b.jpg', 'f82d64a6-3bfa-4…

In [3]:
# Get the list of all jpg files in the folder
jpg_files = [file for file in os.listdir(IMAGE_PATH) if file.lower().endswith('.jpg')]
file_paths = [os.path.join(IMAGE_PATH, file_name) for file_name in jpg_files]

# Print the number of JPG files
print("Number of files:", len(jpg_files))

# Get the details of the first JPG file
if jpg_files:
    first_image_path = os.path.join(IMAGE_PATH, jpg_files[0])
    num_channels, width, height = utils.get_image_details(first_image_path)

    # Print the image details
    print("Image Height:", height)
    print("Image Width:", width)
    print("Number of Channels:", num_channels)
else:
    print("No files found in the folder.")


Number of files: 103
Image Height: 2560
Image Width: 2560
Number of Channels: 3


In [4]:
# Define the path to the "annotations.csv" file
FILE_PATH = config.data_folder
annotations_file_path = os.path.join(FILE_PATH, "annotations.csv")

# Function to convert the "geometry" column to a list of tuples
def convert_geometry(geometry_str):
    return ast.literal_eval(geometry_str)

# Load the CSV file and apply the necessary conversions
annotations_df = pd.read_csv(annotations_file_path, converters={'geometry': convert_geometry, 'class': lambda o: 'Aircraft'})

# Create bounds, width, and height columns in annotations_df
annotations_df['bounds'], annotations_df['width'], annotations_df['height'] = zip(*annotations_df['geometry'].map(utils.calculate_bounds))

# Create a new DataFrame named annotations_df_bounds
annotations_df_bounds = annotations_df.copy()

# Display the updated DataFrame
annotations_df_bounds.head(5)


,id,image_id,geometry,class,bounds,width,height
0,1,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(135, 522), (245, 522), (245, 600), (135, 600...",Aircraft,"(135, 522, 245, 600)",110,78
1,2,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(1025, 284), (1125, 284), (1125, 384), (1025,...",Aircraft,"(1025, 284, 1125, 384)",100,100
2,3,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(1058, 1503), (1130, 1503), (1130, 1568), (10...",Aircraft,"(1058, 1503, 1130, 1568)",72,65
3,4,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(813, 1518), (885, 1518), (885, 1604), (813, ...",Aircraft,"(813, 1518, 885, 1604)",72,86
4,5,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(594, 938), (657, 938), (657, 1012), (594, 10...",Aircraft,"(594, 938, 657, 1012)",63,74


In [5]:
# Create dirs for training and validation data
for split in ['train', 'val']:
    for directory in [config.tiles_dir[split], config.labels_dir[split]]:
        os.makedirs(directory, exist_ok=True)

fold = 1
num_fold = 5
index = annotations_df_bounds['image_id'].unique()

# Calculate the start and end indices for the validation set
start_idx = len(index) * fold // num_fold
end_idx = len(index) * (fold + 1) // num_fold

# Get the validation indexes directly using NumPy slicing
val_indexes = index[start_idx:end_idx]

# Generate tiles for each image
for img_path in tqdm.notebook.tqdm(file_paths):
    utils.generate_tiles(img_path=img_path, df=annotations_df_bounds, val_indexes=val_indexes)

  0%|          | 0/103 [00:00<?, ?it/s]

## YOLO v8

In [ ]:
CONFIG = """
# train and val datasets (image directory or *.txt file with image paths)
train: data/train/images
val: data/val/images

# number of classes
nc: 1

# class names
names: ['Aircraft']
"""

with open("data.yaml", "w") as f:
    f.write(CONFIG)

In [6]:
# !pip install ultralytics

from IPython import display
display.clear_output()
# !yolo checks

In [7]:
HOME = os.getcwd()
!yolo task=detect mode=train model=yolov8s.pt data={HOME}/data.yaml epochs=10 imgsz={config.tile_height}

#  !python /kaggle/working/yolov5/train.py --cfg yolov5s.yaml --imgsz 512 --batch-size 16 --epochs 10 --data /kaggle/working/dataset.yaml --weights yolov5s.pt



Ultralytics YOLOv8.0.147 🚀 Python-3.9.13 torch-2.0.1 CPU (Intel Core(TM) i9-9900K 3.60GHz)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/Users/fabianlow/Documents/4_CV_Application/5_Portfolios/10_AI_DL_SatelliteImages/2_Airplane_Detection/airplane_detection_git/data.yaml, epochs=10, patience=50, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classe

In [ ]:
!ls yolov5/runs/train/exp

In [ ]:
import plotly.express as px
import pandas as pd

df = pd.read_csv("/kaggle/working/runs/detect/train/results.csv")
fig = px.line(df, x='                  epoch', y='       metrics/mAP50(B)', title='mAP50')
fig.show()

In [ ]:
display.Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=1000)

In [ ]:
!rm -rf /kaggle/working/yolov5/runs/detect/
!python /kaggle/working/yolov5/detect.py --source ../input/airbus-aircrafts-sample-dataset/extra s/ --img-size 2560 --weights /kaggle/working/yolov5/runs/train/exp/weights/best.pt --conf 0.5

In [ ]:
csv_file = 'path/to/validation_results.csv'
metrics_to_plot = ['accuracy', 'loss']

plot_metrics_from_csv(csv_file, metrics_to_plot)

## Step 2: Install TensorFlow and TensorFlow Object Detection API

## Step 3: Select a Pre-trained Model (e.g., from the TensorFlow Model Zoo)

In [ ]:
from object_detection.utils import config_util
from object_detection.builders import model_builder

def select_pretrained_model(model_name, num_classes):
    # Load the pre-trained model from the TensorFlow Model Zoo
    pipeline_config_path = "path/to/pretrained_models/{}/pipeline.config".format(model_name)
    configs = config_util.get_configs_from_pipeline_file(pipeline_config_path)
    model_config = configs['model']
    model_config.ssd.num_classes = num_classes

    detection_model = model_builder.build(model_config=model_config, is_training=False)
    return detection_model


## Step 4: Prepare the Configuration

In [ ]:
def create_config_file(dataset_path, num_classes, pretrained_model_path):
    # Create a configuration file using template or API-specific functions
    pipeline_template = """
    model {
      ssd {
        num_classes: %d
        image_resizer {
          fixed_shape_resizer {
            height: 2560
            width: 2560
          }
        }
        feature_extractor {
          type: "ssd_inception_v2"
        }
        box_coder {
          faster_rcnn_box_coder {
            y_scale: 10.0
            x_scale: 10.0
            height_scale: 5.0
            width_scale: 5.0
          }
        }
        anchor_generator {
          ssd_anchor_generator {
            num_layers: 6
            min_scale: 0.2
            max_scale: 0.95
            aspect_ratios: 1.0
            aspect_ratios: 2.0
            aspect_ratios: 0.5
            aspect_ratios: 3.0
            aspect_ratios: 0.3333
          }
        }
        box_predictor {
          convolutional_box_predictor {
            min_depth: 0
            max_depth: 0
            num_layers_before_predictor: 0
            use_dropout: false
            dropout_keep_probability: 0.8
            kernel_size: 1
            box_code_size: 4
            apply_sigmoid_to_scores: false
            conv_hyperparams {
              activation: RELU_6,
              regularizer {
                l2_regularizer {
                  weight: 0.00004
                }
              }
              initializer {
                truncated_normal_initializer {
                  stddev: 0.03
                  mean: 0.0
                }
              }
            }
          }
        }
        image_resizer {
          fixed_shape_resizer {
            height: 2560
            width: 2560
          }
        }
        box_predictor {
          convolutional_box_predictor {
            min_depth: 0
            max_depth: 0
            num_layers_before_predictor: 0
            use_dropout: false
            dropout_keep_probability: 0.8
            kernel_size: 1
            box_code_size: 4
            apply_sigmoid_to_scores: false
            conv_hyperparams {
              activation: RELU_6,
              regularizer {
                l2_regularizer {
                  weight: 0.00004
                }
              }
              initializer {
                truncated_normal_initializer {
                  stddev: 0.03
                  mean: 0.0
                }
              }
            }
          }
        }
      }
    }
    train_config: {
      batch_size: 4
      num_steps: 200000
      fine_tune_checkpoint: "%s"
      fine_tune_checkpoint_type: "detection"
      use_bfloat16: false
      fine_tune_checkpoint_version: V2
      fine_tune_checkpoint_type: "detection"
    }
    train_input_reader: {
      label_map_path: "path/to/label_map.pbtxt"
      tf_record_input_reader {
        input_path: "path/to/training_tfrecord.record"
      }
    }
    eval_config: {
      metrics_set: "coco_detection_metrics"
    }
    eval_input_reader: {
      label_map_path: "path/to/label_map.pbtxt"
      shuffle: false
      num_epochs: 1
      tf_record_input_reader {
        input_path: "path/to/testing_tfrecord.record"
      }
    }
    """ % (num_classes, pretrained_model_path)

    config_file_path = "path/to/your/config_file.config"
    with open(config_file_path, 'w') as config_file:
        config_file.write(pipeline_template)
    
    return config_file_path

# Assuming you have already downloaded a pre-trained model checkpoint from the Model Zoo
pretrained_model_path = "path/to/pretrained_model.ckpt"
num_classes = 1  # Assuming you want to detect one class (e.g., aircraft)
config_file_path = create_config_file(training_dataset_path, num_classes, pretrained_model_path)


## Step 5: Training

In [ ]:
from object_detection import model_lib_v2

def train_model(config_file_path):
    # Start the training process
    model_lib_v2.train_loop(config_override=config_file_path)

# Train the model
train_model(config_file_path)


## Step 6: Evaluation

In [ ]:
from object_detection import model_lib_v2

def evaluate_model(config_file_path):
    # Start the evaluation process
    model_lib_v2.eval_continuously(config_override=config_file_path)

# Evaluate the model
evaluate_model(config_file_path)


## Step 7: Object Detection on New Satellite Images

In [ ]:
def detect_objects(image_path, model):
    # Load the image from the provided path
    image = load_image(image_path)
    
    # Perform object detection on the image using the model
    # You may need to adapt this code based on the API of the selected model
    detections = model.detect_objects(image)
    
    return detections

## Step 8: Visualization

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize_results(image, detections, annotations=None):
    # Visualize the image with bounding boxes and class labels based on the detections
    fig, ax = plt.subplots(1, figsize=(10, 10))
    ax.imshow(image)

    for detection in detections:
        class_id, score, bbox = detection
        x, y, width, height = bbox
        rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(x, y, f'Class: {class_id}, Score: {score:.2f}', fontsize=12, color='r')

    if annotations:
        for annotation in annotations:
            x_coords, y_coords = zip(*annotation)
            rect = patches.Polygon(list(zip(x_coords, y_coords)), linewidth=2, edgecolor='g', facecolor='none')
            ax.add_patch(rect)
    
    plt.show()
